In [19]:
# ! pip install tiktoken
# ! pip install python-dotenv
# ! pip install pandas
# ! pip install pandasql
# ! pip install openai
# ! pip install langchain
# ! pip install langchain\[all\]
# ! pip install pyspark
# ! pip install findspark
# ! pip install faker

# !pip3 install 'mlflow[gateway]'
!pip3 install faiss-cpu

  Using cached faiss_cpu-1.7.4-cp39-cp39-macosx_11_0_arm64.whl (2.7 MB)
You should consider upgrading via the '/Users/saishradhamohanty/Desktop/Repo/Prototypes/.mlflow/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv

import pandas as pd
import warnings
import time
import random



warnings.filterwarnings('ignore')

#### MLFlow Routing Gateway API

In [6]:
# !mlflow gateway start --config-path /Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/monitoring/mlflow_config.yaml!

/Users/saishradhamohanty/Desktop/Repo/Prototypes/.mlflow/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/saishradhamohanty/Desktop/Repo/Prototypes/.mlflow/lib/python3.9/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2023-11-20 20:53:26 -0500] [26532] [INFO] Starting gunicorn 21.2.0
[2023-11-20 20:53:26 -0500] [26532] [INFO] Listening at: http://127.0.0.1:5000 (26532)
[2023-11-20 20:53:26 -0500] [26532] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2023-11-20 20:53:26 -0500] [26533] [INFO] Booting worker with pid: 26533
[2023-11-20 20:53:26 -0500] [26534] [INFO] Booting worker with pid: 26534
/Users/saishradhamohanty

#### Install libraries

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatMLflowAIGateway
from langchain.embeddings import MlflowAIGatewayEmbeddings, AzureOpenAIEmbeddings

In [3]:
import os
import yaml
import json

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.document_loaders import CSVLoader
from langchain.document_loaders.text import TextLoader

In [4]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.vectorstores import FAISS

In [6]:
fallback_message = "Sorry, I'm not intelligent enough to answer your question now. Please try again or with a different question!"

#### Define Azure keys

In [6]:
api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
api_type = 'azure'
model_deployment_name=os.getenv("AZURE_OPENAI_DNAME")

embeddings_deployment_name= os.getenv("AZURE_OPENAI_EMBED_NAME") # os.getenv("AZURE_OPENAI_EMBED_NAME")
embedding_api_version='2022-12-01'
model_api_version='2023-05-15'

max_tokens = 1000
chat_temperature = 0
 #This will correspond to the custom name you chose for your deployment when you deployed a model. 

#### Read schema in JSON format
Why only JSON?

In [7]:
file1 = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/events/schema.csv'

data_file1 = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/sample/data0.csv'
definition_text_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/events/definitions.txt'
fake_data_file = '../askskan/data/original/sample_data.csv'

#### MLFlow variables

In [8]:
mlflow_uri = "http://127.0.0.1:5000"
temperature = 0

#### Load documents

In [9]:
loader_csv = CSVLoader(file_path=file1)
docs_csv = loader_csv.load()

#### Split the documents into chunks

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs_csv)

In [11]:
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
#                               chunk_size=1)

embeddings = AzureOpenAIEmbeddings(
                deployment=embeddings_deployment_name,
                openai_api_key=api_key,
                azure_endpoint=api_endpoint,
                openai_api_type=api_type,
                openai_api_version=embedding_api_version,
                chunk_size=16,
            )

# To use the MLflow Gateway, have to separately query the MLflow server with each document element \
# for obtaining the embeddings
# embeddings = MlflowAIGatewayEmbeddings(
#     gateway_uri=mlflow_uri,
#     route="embeddings",
#     chunk_size=16,
# )

#vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)
vectorstore = FAISS.from_documents(documents, embeddings)


#### Storing verctor_db to a temporary location on local

In [12]:
import tempfile

with tempfile.TemporaryDirectory() as temp_dir:
    persist_dir = os.path.join(temp_dir, "faiss_vectorstore")
    vectorstore.save_local(persist_dir)

#### Function to load the saved vectorstore

In [13]:
def load_retriever(persist_directory, embeddings):
        # embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.load_local(persist_directory, embeddings)
        return vectorstore.as_retriever()

#### Create template to provide as an input to the chain

In [14]:
prompt_without_followups = """
You are a super smart code generator.
Perform the following actions:

1. Understand the question by human, schema, schema definitions and context each delimited by <>.
2. Do not assume any schema attributes unless stated explicitly.
3. Use columns in the schema to generate a final Spark SQL query from the table in data table name delimited by <>\
    answering the original question. The generated query should for data from the start date to end date\
        each delimited by <>. Refer to the schema definitions only for more clarity about terms in the schema.
4. Output computationally most efficient Spark SQL query.
5. Refer to the orginal question again and then select the correct column from the generated Spark SQL query.
5. I have a pandas csv table in data table delimited by <> that contains the data to be queried.
    - Output a valid python code to execute the generated Spark SQL query on the data file.
6. Omit any explanations.

Schema: <{context}>
Schema definitions: <{schema_definitions}>
Data table: <{data_table}>
Data table name: <{data_table_name}>
Start date: <{start_date}>
End date: <{end_date}>


Once the Spark SQL query generated use only the following JSON schema format stricly, refer to the example output:
Output: {{
    "Query": The generated Spark SQL query from the start date to end date,
    "Column": The correct extracted column from the Spark SQL query,
    "Code": Python code generated to run the Spark SQL query. The sql query within this python code should be \
       only be within single quotes e.g. query='SELECT * FROM table' ,
    "Skan Bot": The final answer printed by the python code in a friendly tone. Delimit the answer obtained by \
        python code in ##.
}}

Following is an example of the output:
Output: {{
    "Query": "SELECT app_name, COUNT(*) AS count FROM clipboard WHERE event_date >= '2023-04-01' AND event_date <= '2023-04-30'" \
        GROUP BY app_name ORDER BY count DESC LIMIT 1',
    "Column": "app_name", 
    "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.appName('schema').\
        getOrCreate()\ndata = spark.read.csv('../askskan/data/original/sample_data.csv', header=True, inferSchema=True)\ndata.\
        createOrReplaceTempView('clipboard')\n\nquery = 'SELECT app_name, COUNT(*) AS count FROM clipboard GROUP BY app_name \
        ORDER BY count DESC LIMIT 1'\nresult = spark.sql(query)\n\n spark.stop()\n",
    "Skan Bot": "The most used application is #result#."
}}

nsure the Output can be parsed by Python json.loads
    

Current conversation:
{chat_history}   
Human: <{question}>
Skan Bot: 
"""

#### Questions to be asked by the user

In [15]:
question_1 = "Total time spent on process application?"
question_2 = "Total time spent of non process application?"
question_3 = "Is there an outstanding performer in CES persona?"
question_4 = "Which is the most used application?"
question_5 = "Which is the most active time window during the day?"
question_6 = "What is the average case effort per persona?"
question_7 = "What is the average utilization per persona?"
question_8 = "Which week had highest productivity in CES?"
question_9 = "What is the distribution of time spent in processing application per day?"
question_10 = "What is the average number of participants per case?"
question_11 = "What is the frequency of cases per participant per day?"
question_12 = "Who will be most productive next week?"
question_13 = "Any patterns observed that give an insight on inefficiencies?"
question_14 = "Would taking breaks improve the efficiency in performance?"

question_15 = "What is the standard deviation of time spent on process application?"
question_16 = "What is the standard deviation of time spent on non process application?"

# question_1 = f"""Who has the longest average processing time?"""
# question_2 = f"""Who has the shortest average processing time?"""
# question_3 = f"""Who is the most productive participant based on the number of tasks completed?"""
# question_4 = f"""Who uses the least number of keystrokes on average?"""
# question_5 = f"""Name of the participant that uses the most number of long cut keys on average?""" # An invalid question, to check hallucination
exception_question = f"""How was the day?"""

#### Experimenting with Conversational Retrieval QA

In [16]:
# Specify the path to the text file
file_path = definition_text_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    schema_definitions = file.read()


#### Azure LLM - Now using the one present in the MLFlow API

In [17]:
# Initialize LangChain with Azure OpenAI
chat_llm = AzureChatOpenAI(
    deployment_name=model_deployment_name,
    openai_api_version=model_api_version,
    azure_endpoint=api_endpoint,
    openai_api_key=api_key,
    # max_tokens=max_tokens,
    temperature=0.3,
    streaming=True,
    verbose=True
)

chat_llm_api_version = ChatMLflowAIGateway(
    gateway_uri=mlflow_uri,
    route="chat_35-turbo",
    params={
        "temperature": temperature,
        "streaming": True,
        "verbose": True,
    }
)

In [18]:


# Set up a parser + inject instructions into the prompt template.
# code_parser = PydanticOutputParser(pydantic_object=FormatCodeOutput)
# doubt_parser = PydanticOutputParser(pydantic_object=FormatDoubtOutput)

# new_prompt = PromptTemplate(
#     template=new_template_with_code, 
#     input_variables=["context", "question", "chat_history"] , #"fallback_message"]
#     partial_variables={"schema_definitions": schema_definitions,
#                        "data_table": fake_data_file}
# )

new_prompt = PromptTemplate(
    template=prompt_without_followups,
    input_variables=[
        "context",
        "question",
        "chat_history",
    ],
    partial_variables={"schema_definitions": schema_definitions,
                        "data_table": fake_data_file,
                        "data_table_name": "hive_metastore.unum_askskan.events",
                        "start_date": "2023-04-01",
                        "end_date": "2023-04-30"}
)

chain_type_kwargs = {"prompt": new_prompt}


In [38]:
# Log the chat history separately as a parameter to MLFlow
buffer_memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
qa_api_version = ConversationalRetrievalChain.from_llm(
    llm = chat_llm, # chat_llm_api_version, 
    retriever = vectorstore.as_retriever(), 
    memory=buffer_memory,     # Memory does not get included when logging the model to MLflow
    combine_docs_chain_kwargs=chain_type_kwargs,
    verbose=False,
    get_chat_history=None # lambda h:h
    )

#### Logging the model in MLFLow

In [21]:
import mlflow

#### Persist components


In [39]:
with mlflow.start_run() as run:
        chat_history = lambda h: h
        logged_model = mlflow.langchain.log_model(
            qa_api_version,
            artifact_path="conversational_retrieval_qa_api_version",
            loader_fn=load_retriever,
            persist_dir=persist_dir,
        )

2023/11/24 14:03:49 WARNING mlflow: MLflow does not guarantee support for Chains outside of the subclasses of LLMChain, found ConversationalRetrievalChain


ValueError: Saving of memory is not yet supported.

#### Loading the model from MLFlow before running the predictions

In [ ]:
loaded_model = mlflow.pyfunc.load_model(logged_model.model_uri)

#### Obtaining results

In [ ]:
# result = qa({"question": question_1})
# print(result['answer'])

print(loaded_model.predict([{"question": question_1}]))

{
    "Query": "SELECT SUM(processing_time) AS total_time FROM hive_metastore.unum_askskan.events WHERE agent_type != 0 AND event_date >= '2023-04-01' AND event_date <= '2023-04-30'",
    "Column": "total_time",
    "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.appName('schema').getOrCreate()\ndata = spark.read.csv('../askskan/data/original/sample_data.csv', header=True, inferSchema=True)\ndata.createOrReplaceTempView('hive_metastore.unum_askskan.events')\n\nquery = 'SELECT SUM(processing_time) AS total_time FROM hive_metastore.unum_askskan.events WHERE agent_type != 0 AND event_date >= '2023-04-01' AND event_date <= '2023-04-30''\nresult = spark.sql(query)\n\nspark.stop()\n",
    "Skan Bot": "The total time spent on process applications is #result#."
}


In [ ]:
result = qa({"question": "Where is the Spark SQL query?"})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Total time spent on process application?
AI: Doubt: Are you asking for the total time spent on all process applications combined or for a specific process application?
Human: total time spent on all process applications combined
AI: The total time spent on all process applications combined can be calculated by summing up the processing_time for all process applications. Let me generate the Spark SQL query to calculate it.
Follow Up Input: Where is the Spark SQL query?
Standalone question:


Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
result = qa({"question": "Within a specific day."})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Which is the most active time window during the day?
AI: Doubt: Can you please clarify what you mean by "active time window"? Are you referring to the time of day with the highest active time?
Human: Yes.
AI: Doubt: Can you please clarify what you mean by "highest level of activity"? Are you referring to the time period with the highest total active time?
Human: Yes.
AI: Doubt: Can you please clarify the time range for the active time window? Are you looking for the most active time window within a specific day or across multiple days?
Follow Up Input: Within a specific day.
Standalone question:


Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
code = parser.parse(result['answer']).Code
print(code)

import pandas as pd
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName('ActiveTimeWindow').getOrCreate()

# Read CSV file
df = spark.read.format('csv').option('header', 'true').load('../askskan/data/original/sample_data.csv')

# Register DataFrame as a temporary table
df.createOrReplaceTempView('data_table')

# Generate Spark SQL query
query = "SELECT MAX(active_time) AS max_active_time, CONCAT(HOUR(event_time), ':', MINUTE(event_time)) AS active_time_window FROM data_table GROUP BY CONCAT(HOUR(event_time), ':', MINUTE(event_time)) ORDER BY max_active_time DESC LIMIT 1"

# Execute query
result = spark.sql(query)

# Convert result to Pandas DataFrame
result_df = result.toPandas()

# Print the final answer
print('The most active time window during the day is', result_df['active_time_window'][0], 'with a total active time of', result_df['max_active_time'][0], 'minutes.')


In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [ ]:
retry_parser.parse_with_prompt(code, new_prompt)

AttributeError: 'PromptTemplate' object has no attribute 'to_string'

In [ ]:
result = qa({"question": "I want the output to be in another format."})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Which is the most used application?
AI: Doubt: Can you please clarify what you mean by "most used application"? Are you asking for the application with the highest active time or the application that was switched to the most frequently?
Human: Appliction with highest active time.
AI: Doubt: Are you asking for the application with the highest active time in general or for a specific time period?
Human: In general
AI: Output: The output should be formatted as a JSON instance that conforms to the JSON schema below.

```
{
  "Query": "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1",
  "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.getOrCreate()\ndata = pd.rea

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
result = qa({"question": "I want the output to be in hours and minutes (HH:MM)."})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Which is the most used application?
AI: Doubt: Can you please clarify what you mean by "most used application"? Are you asking for the application with the highest active time or the application that was switched to the most frequently?
Human: Appliction with highest active time.
AI: Doubt: Are you asking for the application with the highest active time in general or for a specific time period?
Human: In general
AI: Output: The output should be formatted as a JSON instance that conforms to the JSON schema below.

```
{
  "Query": "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1",
  "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.getOrCreate()\ndata = pd.rea

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
code = parser.parse(result['answer']).Code
print(code)

import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
data = pd.read_csv('../askskan/data/original/sample_data.csv')
data_table = spark.createDataFrame(data)
data_table.createOrReplaceTempView('data_table')

query = "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1"
result = spark.sql(query)
result.show()



In [ ]:
# result = qa({"question": "processing_time field represents the time taken by the total time taken by a participant for all their tasks"})
# print(result['answer'])

#### Clean the output string to look like JSON

In [ ]:
string = result['answer']

# Find and remove the first occurrence of "Query" in the string
stripped_string = string.replace('Output:', '', 1).strip()

print(stripped_string)


#### Extract the SQL Query

In [ ]:
import json

json_string = result['answer'] # stripped_string

# Remove invalid escape sequences from the JSON string
#json_string = json_string.encode('utf-8').decode('unicode_escape')

# Convert the modified JSON string to a JSON object
json_object = json.loads(json_string, strict=False)

# Extract the "Query" field
query = json_object['Query']

# Print the extracted query
print(query)


SELECT SUM(processing_time) FROM data_table WHERE agent_type = 0


#### Extract the Python code and answer string

In [ ]:
string = """
{"Query": ["SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0"],
"Code": ["import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = \
    SparkSession.builder.appName('schema').getOrCreate()\ndata = spark.read.csv('data/sample/data0.csv', \
        header=True, inferSchema=True)\ndata.createOrReplaceTempView('data_table')\n\nquery = \
            "SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0"\nresult = \
                spark.sql(query)\nresult.show()\nprint('The total time spent on process applications is:', \
                    result.collect()[0][0])"],
"Skan Bot": ["The total time spent on process applications is: #total_time#."],
}"""

In [ ]:
string1 = """
{"Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = \
SparkSession.builder.appName('schema').getOrCreate()\ndata = spark.read.csv('data/sample/data0.csv', \
header=True, inferSchema=True)\ndata.createOrReplaceTempView('data_table')\n\nquery = \
'SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0'\nresult = \
spark.sql(query)\nresult.show()\nprint('The total time spent on process applications is:', \
result.collect()[0][0])"
}"""

In [ ]:
import json

json_string = string1 # result['answer']

# Remove invalid escape sequences from the JSON string
json_string = json_string.encode('utf-8').decode('unicode_escape')

# Find and remove the first occurrence of "Query" in the JSON string
#json_string = json_string.replace('Query', '', 1).strip()

# Convert the modified JSON string to a JSON object
json_object = json.loads(json_string, strict=False)

# Extract the "Query" field
code = json_object['Code']
#answer = json_object['Skan Bot']

# Print the extracted query
print(code)
#print(answer)

import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('schema').getOrCreate()
data = spark.read.csv('data/sample/data0.csv', header=True, inferSchema=True)
data.createOrReplaceTempView('data_table')

query = 'SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0'
result = spark.sql(query)
result.show()
print('The total time spent on process applications is:', result.collect()[0][0])


In [ ]:
try:
    exec(code)
except Exception as e:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import sum

    # Create a SparkSession
    spark = SparkSession.builder.getOrCreate()

    # Load the 'data' table into a DataFrame
    data = spark.table('data')

    # Apply the filter condition
    filtered_data = data.filter(data.agent_type != 0)

    # Calculate the sum of processing_time
    result = filtered_data.select(sum('processing_time').alias('total_time'))

    # Show the result
    result.show()


In [ ]:
import findspark
findspark.init() 

from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

#spark = SparkSession.builder.appName('Query').getOrCreate()

# conf = SparkConf().setAppName("YourAppName").setMaster("local").set("spark.driver.bindAddress", "127.0.0.1")
# spark = SparkContext(conf=conf)
spark = SparkSession.builder.appName("YourAppName").master("local").config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

data = spark.read.format('csv').option('header', 'true').load('../data/sample/data0.csv')
data.createOrReplaceTempView('data0')

query = 'SELECT AVG(processing_time) AS avg_processing_time FROM data'
avg_processing_time = spark.sql(query).collect()[0]['avg_processing_time']

print('The longest average processing time overall is: {} seconds.'.format(avg_processing_time))

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Query').getOrCreate()
data = spark.read.format('csv').option('header', 'true').load('../data/sample/data0.csv')
data.createOrReplaceTempView('data0')

query = 'SELECT participant_id, AVG(processing_time) AS avg_processing_time \
FROM data0 GROUP BY participant_id \
ORDER BY avg_processing_time \
DESC LIMIT 1'

result_df = spark.sql(query).toPandas()
print('The participant with the longest average processing time is:', result_df['participant_id'][0])

In [ ]:
exec(code)

In [ ]:
from azure.identity import DefaultAzureCredential 
from azure.storage.blob import BlobClient

# Storage Account name
account_name = os.environ['DATALAKE_ACCOUNT_NAME']
# Storage Account key
account_key = os.environ['DATALAKE_ACCOUNT_KEY']

# Name of the container where the blob is stored
container_name = "model-training"

# Name of the blob you want to fetch
blob_name = "DataShare/data1/2023-03-01-to-2023-03-31/events/events.csv"

file_path = "../data/original/"
file_name = "events.csv"


account_url = "https://{account_key}.blob.core.windows.net"
# Create a BlobClient object with data transfer options for download
blob_client = BlobClient(
    account_url=account_url, 
    container_name=container_name, 
    blob_name=blob_name,
    credential=account_key, #DefaultAzureCredential(),
    max_single_get_size=1024*1024*32, # 32 MiB
    max_chunk_get_size=1024*1024*4 # 4 MiB
)

with open(file=os.path.join(r'../askskan/data/original/', 'events.csv'), mode="wb") as sample_blob:
    download_stream = blob_client.download_blob(max_concurrency=2)
    sample_blob.write(download_stream.readall())

ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x140512820>: Failed to resolve 'model-training' ([Errno 8] nodename nor servname provided, or not known)